### Setup libraries

In [1]:
# !pip install deeppavlov

### Import libraries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [2]:
import tensorflow as tf

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import io
import re
import time
import unicodedata
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [3]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [4]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

### Paths to directories and files

In [5]:
DRAFT_DATASET_PATH = 'E:/kaggle/mailru/Otvety.txt'

### Loading data

In [6]:
lines = io.open(DRAFT_DATASET_PATH, encoding='UTF-8').read()#.strip().split('\n')

### Preprocessing data

In [7]:
def preprocess_text(w):
    w = re.sub(':\)', '', w)
    w = re.sub('[)"]', '', w)
    w = re.sub('<[^>]+>', ' ', w)
    
    w = re.sub('\s*\?\s*\.', '?', w)
    w = re.sub('\s*\!\s*\.', '!', w)
    w = re.sub('\s*\.', '.', w)
    w = re.sub('\.+', '.', w)
    
#     w = w.lower().strip()
    w = re.sub('---', 'QUESTION', w)
    
    return w

In [8]:
# print(type(lines))
print(len(lines))

1072593541


In [9]:
NUM_EXAMPLES = 500000
text = preprocess_text(str(lines[:NUM_EXAMPLES]))
print(text[:1000])


QUESTION
вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести? 
хомячка. 
мужика, йопаря, собачку и 50 кошек. 
Общение! 
паучка. 
Да пол мне бы памыть! А таг то ни чо. Типа ни каво! 
я тут вообще что бы пообщаться. 
А мне советовали сиси завести. 
Ну, слава богу, мужика завести ещё не советовали А вот сватать к кому только не сватали. 
мне тут советовали завести любовника, мужа и много кошек  приветик. 
QUESTION
Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые. 
меня вобще прикалывает эта тема. 
когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. 
QUESTION
Что делать, сегодня нашёл 2 миллиона рублей? 
Если это счастье  действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни

In [10]:
def split_by_sentence(sent):
    sent = text.split('\nQUESTION\n')[1:]
    
    question = []
    answer = []

    for se in sent:
        se = se.split('\n')
        question.append(se[0].strip())
#         answer.append(' '.join([f'<start_answers> {s}<stop_answers>' for s in se[1:]]))
        answer.append(' '.join([f' {s}' for s in se[1:]]))
    
    return question, answer

In [11]:
question, answer = split_by_sentence(text)
question[0], answer[0]

('вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести?',
 ' хомячка.   мужика, йопаря, собачку и 50 кошек.   Общение!   паучка.   Да пол мне бы памыть! А таг то ни чо. Типа ни каво!   я тут вообще что бы пообщаться.   А мне советовали сиси завести.   Ну, слава богу, мужика завести ещё не советовали А вот сватать к кому только не сватали.   мне тут советовали завести любовника, мужа и много кошек  приветик. ')

In [12]:
df = pd.DataFrame({'questions':question, 'answers':answer})

In [13]:
df.head()

,questions,answers
0,вопрос о ТДВ давно и хорошо отдыхаем ЛИЧНО ВАМ здесь кого советовали завести?,"хомячка. мужика, йопаря, собачку и 50 кошек. Общение! паучка. Да пол мне бы памыть! А таг то ни чо. Типа ни каво! я тут вообще что бы пообщаться. А мне советовали сиси завести. Ну, слава богу, мужика завести ещё не советовали А вот сватать к кому только не сватали. мне тут совет..."
1,"Как парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые.","меня вобще прикалывает эта тема. когда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы."
2,"Что делать, сегодня нашёл 2 миллиона рублей?","Если это счастье действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупк..."
3,Эбу в двенашке называется Итэлма что за эбу?,"ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы. Это завод. А ЭБУ может и..."
4,академия вампиров. сколько на даный момент частей книги академия вампиров?,"4. Охотники и Жертвы, Ледяной укус, Поцелуй тьмы, Кровная клятва. На данное время их 6. часть 5- Оковы для призрака (Духовная связь часть 6-Последняя жертва."


### BERT Question Answering

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

from tokenizers import BertWordPieceTokenizer

In [15]:
class Sample:
    def __init__(self, question, context, start_char_idx=None, answer_text=None, all_answers=None):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False
        self.start_token_idx = -1
        self.end_token_idx = -1

    def preprocess(self):
        # clean context and question
        context = " ".join(str(self.context).split())
        question = " ".join(str(self.question).split())
        # tokenize context and question
        tokenized_context = tokenizer.encode(context)
        tokenized_question = tokenizer.encode(question)
        # if this is validation or training sample, preprocess answer
        if self.answer_text is not None:
            answer = " ".join(str(self.answer_text).split())
            # check if end character index is in the context
            end_char_idx = self.start_char_idx + len(answer)
            if end_char_idx >= len(context):
                self.skip = True
                return
            # mark all the character indexes in context that are also in answer     
            is_char_in_ans = [0] * len(context)
            for idx in range(self.start_char_idx, end_char_idx):
                is_char_in_ans[idx] = 1
            ans_token_idx = []
            # find all the tokens that are in the answers
            for idx, (start, end) in enumerate(tokenized_context.offsets):
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
            if len(ans_token_idx) == 0:
                self.skip = True
                return
            # get start and end token indexes
            self.start_token_idx = ans_token_idx[0]
            self.end_token_idx = ans_token_idx[-1]
        # create inputs as usual
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        # add padding if necessary
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            self.skip = True
            return
        self.input_word_ids = input_ids
        self.input_type_ids = token_type_ids
        self.input_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets

In [16]:
# def create_squad_examples(raw_data):
#     squad_examples = []
#     for item in raw_data["data"]:
#         for para in item["paragraphs"]:
#             context = para["context"]
#             for qa in para["qas"]:
#                 question = qa["question"]
#                 if "answers" in qa:
#                     answer_text = qa["answers"][0]["text"]
#                     all_answers = [_["text"] for _ in qa["answers"]]
#                     start_char_idx = qa["answers"][0]["answer_start"]
#                     squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
#                 else:
#                     squad_eg = Sample(question, context)
#                 squad_eg.preprocess()
#                 squad_examples.append(squad_eg)
#     return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_word_ids": [],
        "input_type_ids": [],
        "input_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
#         if item.skip == False:
        if item == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
    x = [dataset_dict["input_word_ids"],
         dataset_dict["input_mask"],
         dataset_dict["input_type_ids"]]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

In [17]:
class ValidationCallback(keras.callbacks.Callback):

    def normalize_text(self, text):
        # convert to lower case
        text = text.lower()
        # remove redundant whitespaces
        text = "".join(ch for ch in text if ch not in set(string.punctuation))
        # remove articles
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        text = re.sub(regex, " ", text)
        text = " ".join(text.split())
        return text

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        # get the offsets of the first and last tokens of predicted answers
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        # for every pair of offsets
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            # take the required Sample object with the ground-truth answers in it
            squad_eg = eval_examples_no_skip[idx]
            # use offsets to get back the span of text corresponding to
            # our predicted first and last tokens
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]
            normalized_pred_ans = self.normalize_text(pred_ans)
            # clean the real answers
            normalized_true_ans = [self.normalize_text(_) for _ in squad_eg.all_answers]
            # check if the predicted answer is in an array of the ground-truth answers
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch + 1}, exact match score={acc:.2f}")

In [19]:
# train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
# eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
# with open(train_path) as f: raw_train_data = json.load(f)
# with open(eval_path) as f: raw_eval_data = json.load(f)
raw_train_data = text
raw_eval_data = text
max_seq_length = 384

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy().decode("utf-8")
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertWordPieceTokenizer(vocab=vocab_file, lowercase=True)

train_squad_examples = raw_train_data
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = raw_eval_data
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

start_logits = layers.Dense(1, name="start_logit", use_bias=False)(sequence_output)
start_logits = layers.Flatten()(start_logits)
end_logits = layers.Dense(1, name="end_logit", use_bias=False)(sequence_output)
end_logits = layers.Flatten()(end_logits)
start_probs = layers.Activation(keras.activations.softmax)(start_logits)
end_probs = layers.Activation(keras.activations.softmax)(end_logits)

model = keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=[start_probs, end_probs])
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer=optimizer, loss=[loss, loss])
model.summary()
model.fit(x_train, y_train, epochs=2, batch_size=1)#, callbacks=[ValidationCallback(x_eval, y_eval)])
model.save_weights("./weights.h5")

486080 training points created.
486080 evaluation points created.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 384)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                          

ValueError: Expect x to be a non-empty array or dataset.

https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626

In [21]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [24]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [25]:
random_num = np.random.randint(0,len(df))
question = df["questions"][random_num]
text = df["answers"][random_num]

In [26]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 327 tokens.


In [27]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
в          1,182
##о       14,150
##п       29,746
##р       16,856
##о       14,150
##с       29,747
о          1,193
т          1,197
##д       29,742
##в       25,529
д          1,184
##а       10,260
##в       25,529
##н       18,947
##о       14,150
и          1,188
х          1,200
##о       14,150
##р       16,856
##о       14,150
##ш       29,753
##о       14,150
о          1,193
##т       22,919
##д       29,742
##ы       29,113
##х       29,750
##а       10,260
##е       15,290
##м       29,745
л          1,190
##и       10,325
##ч       29,752
##н       18,947
##о       14,150
в          1,182
##а       10,260
##м       29,745
з          1,187
##д       29,742
##е       15,290
##с       29,747
##ь       23,742
к          1,189
##о       14,150
##г       29,741
##о       14,150
с          1,196
##ов      19,259
##е       15,290
##т       22,919
##ов      19,259
##а       10,260
##л       29,436
##и       10,325
з          1,187
##а       10,260
##в       25,5

In [28]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  64
Number of tokens in segment A:  65
Number of tokens in segment B:  262


In [29]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),  token_type_ids=torch.tensor([segment_ids]))

In [31]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
Вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести?

Answer:
[cls].
